In [4]:
import os
import pickle

from params import *
from utils.read2df import read2df, unify_dfs

In [5]:
with open('result/cointncorr.pickle', 'rb') as pk:
    data = pickle.load(pk)

bestres_pickle = 'result/gridsearch/best_res.pickle'
if os.path.exists(bestres_pickle):
    with open(bestres_pickle, 'rb') as pk:
        best_profit, best_params = pickle.load(pk)
else:
    print("pickle is not found, please execute `trade_gridsearch.ipynb` first")

dfs = read2df(symbols=data[0], freqs={data[1]: freqs[data[1]]})

df = unify_dfs(dfs, symbols=data[0], period=best_params['period'])

In [3]:
df

,time,close_ETHEUR,itvl,datetime,close_ETHGBP,spread,zscore
0,1592560859999,0.004827,1m,2020-06-19 10:00:59.999,0.005422,-0.000594,0.000000
1,1592560919999,0.004827,1m,2020-06-19 10:01:59.999,0.005412,-0.000584,0.000000
2,1592560979999,0.004827,1m,2020-06-19 10:02:59.999,0.005412,-0.000584,0.000000
3,1592561039999,0.004854,1m,2020-06-19 10:03:59.999,0.005412,-0.000557,0.000000
4,1592561099999,0.004854,1m,2020-06-19 10:04:59.999,0.005412,-0.000557,0.000000
...,...,...,...,...,...,...,...
1812156,1701388559999,0.000530,1m,2023-11-30 23:55:59.999,0.000615,-0.000084,-0.628750
1812157,1701388619999,0.000530,1m,2023-11-30 23:56:59.999,0.000615,-0.000084,-0.603341
1812158,1701388679999,0.000530,1m,2023-11-30 23:57:59.999,0.000615,-0.000084,-0.623742
1812159,1701388739999,0.000530,1m,2023-11-30 23:58:59.999,0.000615,-0.000084,-0.613500
